# Importamos librerias 


In [346]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize


# Carga de los datasets


In [ ]:
movies_df = pd.read_csv(r"C:\Users\Jonatan Sivisstum\Downloads\Movies-20241217T205003Z-001\Movies\movies_dataset.csv")
credits_df = pd.read_csv(r"C:\Users\Jonatan Sivisstum\Downloads\Movies-20241217T205003Z-001\Movies\credits.csv")


In [ ]:
movies_df.info()

# Eliminamos las columnas que no serán utilizadas

In [349]:
# Columnas a eliminar
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']

# Eliminamos las columnas del DataFrame
movies_df.drop(columnas_a_eliminar, axis=1, inplace=True)

# RELLENAMOS LOS VALORES NULOS POR EL NUMERO 0.

In [350]:
# Rellenamos los valores nulos de las columnas 'revenue' y 'budget' con 0
movies_df[['revenue', 'budget']] = movies_df[['revenue', 'budget']].fillna(0)


 # ELIMINAMOS LOS VALORES NULOS DEL CAMPO "RELEASE".

In [351]:
# Eliminamos las filas donde 'release_date' es nulo
movies_df.dropna(subset=['release_date'], inplace=True)

# Pasamos la columna de fechas a formato AAAA-mm-dd

Convertimos la columna 'release_date' a formato datetime

In [352]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

# Creamos la columna "release_year" donde indica el año de la fecha de estreno.

In [353]:
movies_df['release_year'] = movies_df['release_date'].dt.year

In [415]:
# Convertir los valores de 'release_year' a enteros y rellenar valores nulos con 0
movies_df['release_year'] = movies_df['release_year'].fillna(0).astype(int)


In [411]:
movies_df.dropna(subset=['release_date'], inplace=True)


# Creamos la columna con retorno de inversión, llamada "return"

cuando no hay datos disponibles,tomará el valor 0.

In [354]:
# Convertimos las columnas 'revenue' y 'budget' a numéricas, manejando errores
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')

# Creamos la columna 'return' calculando el ROI, manejando divisiones por cero
movies_df['return'] = np.where(movies_df['budget'] > 0, movies_df['revenue'] / movies_df['budget'], 0)

In [ ]:
movies_df['budget'].fillna(0, inplace=True)


# Eliminamos columnas irrelevantes


In [355]:
movies_df = movies_df.drop('tagline', axis=1)

In [ ]:
datos_nulos = movies_df.isnull().sum()
print(datos_nulos)

In [ ]:
movies_df.describe()

Eliminamos filas repetidas del dataset.

In [358]:
movies_df.drop_duplicates(inplace=True)

# DESANIDAMOS LAS COLUMNAS.

Desanidamos la columna "belongs_to_collection"

In [ ]:
# Función para procesar cada celda de manera segura
def safe_json_load(cell):
    try:
        # Reemplazar comillas simples por dobles si es una cadena
        if isinstance(cell, str):
            cell = cell.replace("'", '"')
        # Intentar convertir a JSON
        return json.loads(cell)
    except (json.JSONDecodeError, TypeError):
        # Devolver None si falla la conversión
        return None

# Desanidamos la columna "belongs_to_collection"
movies_df['belongs_to_collection'] = movies_df['belongs_to_collection'].apply(safe_json_load)

# Extraemos las claves relevantes del diccionario (si existen)
movies_df['collection_id'] = movies_df['belongs_to_collection'].apply(
    lambda x: x['id'] if isinstance(x, dict) else None
)
movies_df['collection_name'] = movies_df['belongs_to_collection'].apply(
    lambda x: x['name'] if isinstance(x, dict) else None
)
movies_df['collection_poster_path'] = movies_df['belongs_to_collection'].apply(
    lambda x: x['poster_path'] if isinstance(x, dict) else None
)
movies_df['collection_backdrop_path'] = movies_df['belongs_to_collection'].apply(
    lambda x: x['backdrop_path'] if isinstance(x, dict) else None
)

# Verificamos los primeros registros
movies_df[['belongs_to_collection', 'collection_id', 'collection_name']].head()


In [ ]:
# Verificar si hay valores nulos en la columna 'belongs_to_collection' y en las columnas desanidadas
null_values = movies_df[['belongs_to_collection', 'collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path']].isnull().sum()

# Verificar el tipo de datos de las nuevas columnas
column_types = movies_df[['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path']].dtypes

# Verificar cuántas filas tienen 'None' en la columna 'belongs_to_collection'
none_rows = movies_df[movies_df['belongs_to_collection'].isna()]

# Mostrar resultados
print("Valores nulos en las columnas:")
print(null_values)
print("\nTipos de datos de las nuevas columnas:")
print(column_types)
print("\nFilas con 'None' en 'belongs_to_collection':")
print(none_rows.head())  # Muestra solo las primeras filas con 'None'


In [ ]:
# Reemplazamos NaN y None con un valor específico para indicar que no pertenece a una colección
movies_df['collection_name'].fillna('No tiene colección', inplace=True)
movies_df['collection_id'].fillna(-1, inplace=True)  # Usamos -1 para los ids sin colección
movies_df['collection_poster_path'].fillna('No tiene poster', inplace=True)
movies_df['collection_backdrop_path'].fillna('No tiene backdrop', inplace=True)

# Verificamos los cambios
movies_df[['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path']].head()


In [ ]:
# Filas problemáticas
invalid_rows = movies_df[movies_df['belongs_to_collection'].isna()]
print(f"Filas con problemas de desanidado: {len(invalid_rows)}")



Eliminamos la columna "belongs_to_collection"

In [363]:
movies_df.drop(columns=['belongs_to_collection'], inplace=True)


# Desanidamos la columna "genres"


In [ ]:
import ast

# Convertir las cadenas a listas de diccionarios
movies_df['genres'] = movies_df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer los nombres de los géneros
movies_df['genre_names'] = movies_df['genres'].apply(
    lambda x: ', '.join([genre['name'] for genre in x]) if isinstance(x, list) else None
)

# Verifica las primeras filas
movies_df[['genres', 'genre_names']].head()


In [ ]:
# Filas con problemas de desanidado
invalid_genres = movies_df[movies_df['genre_names'].isna()]
print(f"Filas con problemas en la columna 'genres': {len(invalid_genres)}")


Eliminamos la columna "genres"

In [366]:
movies_df.drop('genres', axis=1, inplace=True)


# Desanidamos la columna "production_companies "

In [367]:
import json

# Función para limpiar y cargar el JSON de forma segura
def clean_and_load_json(x):
    if isinstance(x, str):
        # Reemplazar comillas simples por dobles
        x = x.replace("'", '"')
        try:
            # Intentar cargar el JSON
            return json.loads(x)
        except json.JSONDecodeError:
            # Si falla, devolver el valor original
            return None
    return x

# Aplicar la limpieza y carga
movies_df['production_companies'] = movies_df['production_companies'].apply(clean_and_load_json)

# Extraer los nombres de las compañías de producción
movies_df['production_companies_names'] = movies_df['production_companies'].apply(
    lambda x: ', '.join([company['name'] for company in x]) if isinstance(x, list) else None
)

# Extraer los ids de las compañías de producción
movies_df['production_companies_ids'] = movies_df['production_companies'].apply(
    lambda x: ', '.join([str(company['id']) for company in x]) if isinstance(x, list) else None
)

# Verifica los primeros registros para confirmar que los cambios se hicieron correctamente
movies_df[['production_companies', 'production_companies_names', 'production_companies_ids']].head()

# Opcional: Eliminar la columna original 'production_companies' si ya no es necesaria
movies_df.drop('production_companies', axis=1, inplace=True)




In [ ]:
# Rellenar los valores None con 'Desconocido'
movies_df['production_companies_names'].fillna('Desconocido', inplace=True)


In [ ]:
# Verificar si hay filas con problemas de desanidado
invalid_companies = movies_df[movies_df['production_companies_names'].isna()]
print(f"Filas con problemas en la columna 'production_companies': {len(invalid_companies)}")


Eliminamos la columna "production_companies"

In [376]:
movies_df.drop('production_companies', axis=1, inplace=True, errors='ignore')


# Desanidamos la columna "production_countries"

In [ ]:
# Verificamos los primeros valores de la columna 'production_countries'
movies_df['production_countries'].head()

# Intentamos reemplazar las comillas simples por dobles y manejar excepciones
def safe_json_load(x):
    try:
        # Reemplazamos comillas simples por dobles y tratamos de cargarlo como JSON
        return json.loads(x.replace("'", '"')) if isinstance(x, str) else x
    except json.JSONDecodeError:
        # Si ocurre un error, devolvemos el valor original
        return x

# Aplicamos la función para manejar errores de formato
movies_df['production_countries'] = movies_df['production_countries'].apply(safe_json_load)

# Extraemos los nombres de los países
movies_df['production_countries_names'] = movies_df['production_countries'].apply(
    lambda x: ', '.join([country['name'] for country in x]) if isinstance(x, list) else None
)

# Extraemos los códigos de los países (iso_3166_1)
movies_df['production_countries_codes'] = movies_df['production_countries'].apply(
    lambda x: ', '.join([country['iso_3166_1'] for country in x]) if isinstance(x, list) else None
)

# Mostrar las nuevas columnas
movies_df[['production_countries', 'production_countries_names', 'production_countries_codes']].head()


Eliminamos la columna "production_countries"

In [379]:
movies_df.drop('production_countries', axis=1, inplace=True)


In [ ]:
movies_df['production_countries_names'].fillna('Desconocido', inplace=True)
movies_df['production_countries_codes'].fillna('Desconocido', inplace=True)


In [ ]:
print(movies_df[['production_countries_names', 'production_countries_codes']].isna().sum())


Desanidamos la columna "spoken_languages"

In [ ]:
import ast

# Usamos ast.literal_eval para evitar problemas con caracteres especiales o comillas mal formateadas
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Extraemos los nombres de los idiomas
movies_df['spoken_languages_names'] = movies_df['spoken_languages'].apply(
    lambda x: ', '.join([language['name'] for language in x]) if isinstance(x, list) else None
)

# Extraemos los códigos de los idiomas
movies_df['spoken_languages_codes'] = movies_df['spoken_languages'].apply(
    lambda x: ', '.join([language['iso_639_1'] for language in x]) if isinstance(x, list) else None
)


Eliminamos la columna "spoken_languages"

In [ ]:
movies_df.drop('spoken_languages', axis=1, inplace=True)


In [ ]:
import pandas as pd

# Establecer la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar el DataFrame
print(movies_df)


In [ ]:
invalid_languages = movies_df[movies_df['spoken_languages_names'].isna()]
print(invalid_languages[['spoken_languages_names', 'spoken_languages_codes']].head())


In [ ]:
movies_df['spoken_languages_names'].fillna('Desconocido', inplace=True)
movies_df['spoken_languages_codes'].fillna('Desconocido', inplace=True)


In [ ]:
movies_df[['spoken_languages_names', 'spoken_languages_codes']].isna().sum()


# Configuramos el Proyecto con FastAPI


Instalación de dependencias


In [ ]:
pip install fastapi uvicorn


# Configuramos FastAPI

In [427]:
from fastapi import FastAPI


# Inicializar FastAPI
app = FastAPI()

# Endpoint para "cantidad_filmaciones_mes"

In [428]:
@app.get("/cantidad_filmaciones_mes/{mes}")
def cantidad_filmaciones_mes(mes: str):
    # Mapa de meses en español a números
    meses_espanol = {
        "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
        "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
        "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
    }
    
    mes_numero = meses_espanol.get(mes.lower(), None)
    if mes_numero is None:
        return {"error": "Mes no válido"}
    
    # Filtrar las películas que se estrenaron en el mes
    peliculas_mes = movies_df[movies_df['release_date'].dt.month == mes_numero]
    return {"mensaje": f"{len(peliculas_mes)} cantidad de películas fueron estrenadas en el mes de {mes}"}


# Endpoint para "cantidad_filmaciones_dia"


In [429]:
@app.get("/cantidad_filmaciones_dia/{dia}")
def cantidad_filmaciones_dia(dia: str):
    # Mapa de días en español a números
    dias_espanol = {
        "lunes": 0, "martes": 1, "miércoles": 2, "jueves": 3,
        "viernes": 4, "sábado": 5, "domingo": 6
    }
    
    dia_numero = dias_espanol.get(dia.lower(), None)
    if dia_numero is None:
        return {"error": "Día no válido"}
    
    # Filtrar las películas que se estrenaron en el día de la semana
    peliculas_dia = movies_df[movies_df['release_date'].dt.weekday == dia_numero]
    return {"mensaje": f"{len(peliculas_dia)} cantidad de películas fueron estrenadas en los días {dia}"}


# Endpoint para "score_titulo"


In [430]:
@app.get("/score_titulo/{titulo_de_la_filmacion}")
def score_titulo(titulo_de_la_filmacion: str):
    pelicula = movies_df[movies_df['title'].str.contains(titulo_de_la_filmacion, case=False, na=False)]
    if pelicula.empty:
        return {"error": "Película no encontrada"}
    pelicula = pelicula.iloc[0]
    return {
        "mensaje": f"La película {pelicula['title']} fue estrenada en el año {pelicula['release_year']} con un score/popularidad de {pelicula['popularity']}"
    }
